In [282]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [283]:

class Model(nn.Module):

    def __init__(self, in_features = 4, h1 = 8, h2 = 8, out_features = 3):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1 , h2)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x) :
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x
    

In [284]:
# torch.manual_seed(41)
model = Model()

In [285]:
import matplotlib.pyplot as plt
import pandas as pd

In [286]:
df = pd.read_csv("irisDataset/iris.csv")
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [287]:
# Cleansing
df["species"] = df["species"].replace("setosa", 0.0)
df["species"] = df["species"].replace("versicolor", 1.0)
df["species"] = df["species"].replace("virginica", 2.0)

C:\Users\izuha\AppData\Local\Temp\ipykernel_16656\2289834928.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["species"] = df["species"].replace("virginica", 2.0)


In [288]:
x = df.drop("species", axis=1)
y = df["species"]

# convert to numpy arr
x = x.values 
y = y.values


In [289]:
from sklearn.model_selection import train_test_split

x_train , x_test  , y_train , y_test = train_test_split(x, y, test_size= 0.2)
y_test

array([0., 2., 1., 0., 0., 0., 1., 2., 2., 0., 0., 0., 2., 1., 0., 1., 1.,
       0., 1., 1., 0., 1., 2., 1., 2., 1., 2., 1., 2., 2.])

In [290]:
#  convert to tensors
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)

In [291]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters() , lr = 0.01)

In [292]:
## Train model 

epochs = 200
losses = []
for i in range(epochs):

    # forward prop
    y_pred = model.forward(x_train)

    # cose fun
    loss = criterion(y_pred, y_train)

    if i % 10 == 0 : 
        print(f'Epoch: {i} and loss: {loss}')
    
    # back prop
    optimizer.zero_grad()
    loss.backward() 
    optimizer.step()

Epoch: 0 and loss: 1.1746485233306885
Epoch: 10 and loss: 0.9042167663574219
Epoch: 20 and loss: 0.6416109800338745
Epoch: 30 and loss: 0.45082515478134155
Epoch: 40 and loss: 0.3310324251651764
Epoch: 50 and loss: 0.2283850610256195
Epoch: 60 and loss: 0.14799997210502625
Epoch: 70 and loss: 0.10181091725826263
Epoch: 80 and loss: 0.07862813770771027
Epoch: 90 and loss: 0.0663895383477211
Epoch: 100 and loss: 0.059065598994493484
Epoch: 110 and loss: 0.05414421111345291
Epoch: 120 and loss: 0.050545983016490936
Epoch: 130 and loss: 0.04773951321840286
Epoch: 140 and loss: 0.04543866589665413
Epoch: 150 and loss: 0.04347122460603714
Epoch: 160 and loss: 0.04172907769680023
Epoch: 170 and loss: 0.04014469310641289
Epoch: 180 and loss: 0.03867434710264206
Epoch: 190 and loss: 0.037288833409547806


In [293]:
## Test model on x_test

counter = 0
with torch.no_grad():
    for i, data in enumerate(x_test):

        y_val = model.forward(data)
        print(f'{i}.) {y_val} \t {y_val.argmax().item()} \t {y_test[i]}')

        if y_val.argmax().item() == y_test[i]:
            counter += 1
        
    print(f'number of correct things is {counter} ')

0.) tensor([ 13.4341,   5.3504, -25.0590]) 	 0 	 0
1.) tensor([-6.9476,  3.2391,  1.0860]) 	 1 	 2
2.) tensor([-7.0858,  3.3408,  1.0492]) 	 1 	 1
3.) tensor([ 15.3011,   5.8016, -28.1605]) 	 0 	 0
4.) tensor([ 14.0769,   5.3099, -25.8587]) 	 0 	 0
5.) tensor([ 13.7596,   5.3401, -25.4619]) 	 0 	 0
6.) tensor([-2.0040,  5.7513, -7.4105]) 	 1 	 1
7.) tensor([-11.8764,   1.2195,   8.7565]) 	 2 	 2
8.) tensor([-9.3632,  1.2569,  6.4711]) 	 2 	 2
9.) tensor([ 11.0133,   5.0252, -21.5401]) 	 0 	 0
10.) tensor([ 15.2004,   5.5166, -27.6601]) 	 0 	 0
11.) tensor([ 16.5806,   5.7375, -29.8342]) 	 0 	 0
12.) tensor([-11.6651,  -0.1678,  10.4124]) 	 2 	 2
13.) tensor([-4.1331,  3.7592, -2.3319]) 	 1 	 1
14.) tensor([ 16.5027,   5.8291, -29.8453]) 	 0 	 0
15.) tensor([-3.7788,  5.6207, -5.6018]) 	 1 	 1
16.) tensor([-4.1752,  4.2990, -3.1412]) 	 1 	 1
17.) tensor([ 16.2802,   5.5709, -29.2090]) 	 0 	 0
18.) tensor([-4.5324,  3.9003, -2.1505]) 	 1 	 1
19.) tensor([-3.2357,  5.1461, -5.3305]) 	 1 	

In [294]:
# Mock data point

new_iris = torch.tensor([4.7, 3.2, 1.3, 0.2])

with torch.no_grad() :
    ypred = model.forward(new_iris)

    print(ypred.argmax().item())

0


In [295]:
## Save model

torch.save(model.state_dict() , 'irisModel.pt')